### Performing exploratory data analysis on data from the StakeOverFlow developer survey from the years 2018 to 2021

##### The purpose of this notebook is to analyze StackOverflow Developer Survey data in order to answer the following questions:

##### Question 1. *How much impact has the pandemic had on developer's choices of tech stack?*
This is question is broken down to three parts namely:
- Question 1.1 Which programming languages have gained popularity from 2018 to 2021?
- Question 1.2 Which database services have gained popularity from 2018 to 2021?
- Question 1.3 Which platforms and frameworks have gained popularity from 2018 to 2021?
##### Question 2. *How has the distribution of gender and age in the developer community changed from 2018 to 2021?*

#### Table of Contents

Extracting the data\
Data Model\
Transforming the data
- Cleaning the data
- Merging dataframes
- Cleaning the new dataframe

##### Phase 3 : Loading the data
- Loading in the data into a database

## Extracting the data

While writing this notebook, the data was stored locally in my computer therefore to follow along with this note book you will need to download the data from here : https://insights.stackoverflow.com/survey/

In [6]:
import json
import logging
import numpy as np
import pandas as pd
from sqlalchemy import create_engine 

#setting the maximum display for the notebook cells 
pd.set_option('display.max_rows', 48)
pd.set_option('display.max_columns', 48)

#removing warnings 
pd.options.mode.chained_assignment = None  # default='warn'


def get_credentials(filepath : str) -> dict:
    """Loads database credentials from file.
    Args: 
        filepath - path to the json file

    Returns :
        A dictionary containing database credentials
    """
    with open(filepath, "r") as file:
        data = json.loads(file.read())
   
    return data

credentials = get_credentials("..\sof_sa\conf\staging_db_credentials.json")

def get_data_from_db(path_to_sql_file: str, credentials: dict) -> pd.DataFrame:
    """Executes an sql query 

    Args:
        path_to_sql_file (str): path to the sql file that contains the sql statement to execute.
        credentials (dict): credentials to the database where the query will be executed 

    Returns:
        pd.DataFrame: a pandas dataframe representing the results of the query
    """
    try:
        DATABASE_URL = f'postgresql+psycopg2://{credentials["user"]}:{credentials["password"]}@{credentials["host"]}:{credentials["port"]}/{credentials["database"]}'
        engine = create_engine(DATABASE_URL, pool_pre_ping=True)

        with open(path_to_sql_file, 'r') as file, engine.connect() as connection:
            df = pd.read_sql_query(file.read(), connection)
            return df
    except Exception as e:
        logging.error(e)
    
df2018 = get_data_from_db("..\sof_sa\SQL\select_2018_data.sql", credentials)
df2019 = get_data_from_db("..\sof_sa\SQL\select_2019_data.sql", credentials)
df2020 = get_data_from_db("..\sof_sa\SQL\select_2020_data.sql", credentials)
df2021 = get_data_from_db("..\sof_sa\SQL\select_2021_data.sql", credentials)

#### Data Modelling

![Data Model](../img/model.jpg)

## Transforming the data

In [7]:
# The size of the different datasets
print(f"df2018 has shape : {df2018.shape}")
print(f"df2019 has shape : {df2019.shape}")
print(f"df2020 has shape : {df2020.shape}")
print(f"df2021 has shape : {df2021.shape}")

df2018 has shape : (98855, 12)
df2019 has shape : (88883, 13)
df2020 has shape : (64461, 13)
df2021 has shape : (83439, 13)


In [8]:
# question 1
from collections import Counter

def count_unique_items_in_column(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """Counts unique elements in dataframe column. Column must have semicolon separated values or nan values in column

    Args:
        df (pd.DataFrame): dataframe to be modified
        column_name (str): column name in dataframe

    Returns:
        pd.DataFrame: new dataframe contain value and count of value in df
    
    Raises:
        ValueError: if the column passed does not exist in dataframe
    """
    if not column_name in df.columns:
        raise ValueError(f"No column named {column_name} in dataframe.")
    
    column_as_list = df[column_name].tolist()
    
    new_list = []
    for list_item in column_as_list: 
        
        # for nan values
        if isinstance(list_item, type(None)):
            new_list.append(list_item)
            
        if isinstance(list_item, str): 
            new_list.extend(list_item.split(";"))
            
    # find the number of occurances of a item in a list
    occ = Counter(new_list)
    language = []
    count = []
    for x in occ:
        key = x
        value = occ[key]
        language.append(key)
        count.append(value)

    df_temp = pd.DataFrame(list(zip(language, count)), columns = [column_name, 'count'])
    df_temp.set_index(column_name, inplace=True)
    df_temp.sort_values(by='count', ascending=False, inplace=True)
    return df_temp


def merge_dfs(dataframe_list: list, column_name: str) -> pd.DataFrame:
    """Merges dataframes on column_name

    Args:
        dataframe_list (list): a list of dataframes to merge
        column_name (str): column name to merge dataframes on 

    Returns:
        pd.DataFrame: merged dataframe
        
    Raises:
        ValueError: if the list of dataframes passed is not equal to four
    """
    if len(dataframe_list) != 4:
        raise ValueError("List of dataframes must be equal to four(4)")
     
    dfs = []
    for df in dataframe_list:
        df1 = count_unique_items_in_column(df, column_name)
        dfs.append(df1)
        
    df18_19 = pd.merge(dfs[0], dfs[1], on=column_name)
    df20_21 = pd.merge(dfs[2], dfs[3], on=column_name)
    dfs_merged = pd.merge(df18_19, df20_21, on=column_name)
    dfs_merged.columns = ['2018', '2019', '2020', '2021']
    
    return dfs_merged


In [9]:
dataframes = [df2018, df2019, df2020, df2021]
# Languages
languages = merge_dfs(dataframes, 'language_worked_with')
future_languages = merge_dfs(dataframes, 'language_desire_next_year')

# databases
databases = merge_dfs(dataframes, 'database_worked_with')
future_databases = merge_dfs(dataframes, 'database_desire_next_year')

# platforms
platforms = merge_dfs(dataframes, 'platform_worked_with')
future_platforms = merge_dfs(dataframes, 'platform_desire_next_year')

# web_frameworks
web_frameworks = merge_dfs(dataframes, 'web_framework_have_worked_with')
future_web_frameworks = merge_dfs(dataframes, 'web_framework_want_to_work_with')

In [49]:
future_web_frameworks

,2018,2019,2020,2021
web_framework_want_to_work_with,,,,
None,43637,25939,24437,31344
Django,8774,11358,8237,9112
Spring,8063,9846,6241,7229


In [10]:
# question 2

def age_to_range(number: int) -> str:
    """Checks if a certain value falls within a certain range then retruns the appropriate string

    Args:
        number (int): number to be checked

    Returns:
        str: a string based on the number passed
    """

    if number < 18:
        return 'Under 18 years old'
    elif number >= 18 and number <= 24:
        return '18 - 24 years old'
    elif number >= 25 and number <= 30:
        return '25 - 30 years old'
    elif number >= 31 and number <= 36:
        return '31 - 36 years old'
    elif number >= 37 and number <= 42:
        return '37 - 42 years old'
    elif number >= 43 and number <= 48:
        return '43 - 48 years old'
    elif number >= 49 and number <= 54:
        return '49 - 54 years old'
    elif number >= 55 and number <= 60:
        return '55 - 60 years old'
    elif number > 60:
        return 'Over 60 years old'

def clean_age_column(age) -> str:
    """Cleans the age column of a dataframe

    Args:
        age (Any): An int, str or float representing age

    Returns:
        str: a string based on the age passed
    """
    if isinstance(age, str):
        n = age.replace(" ", "")
        if 'or' in n:
            return age_to_range(int(n[0:2]))
                
        if 'Under' in n:
            return age_to_range(int(n[5:7]))
            
        if '-' in n:
            return age_to_range((int(n[0:2]) + int(n[3:5]))//2)
                
        if 'Prefer' in n:
            return 'Prefer not to say'
        
        if n is None:
            return 'Prefer not to say'
            
    if isinstance(age, float) or isinstance(age, int):
        return age_to_range(round(age))

def replace_na_with_mean(df: pd.DataFrame, column_name: str) -> None:
    """Replaces na values in column of a dataframe with mean

    Args:
        df (pd.DataFrame): dataframe to be modified
        column_name (str): column in dataframe
    
    Raises:
        ValueError: if the column passed does not exist in dataframe
    """
    
    if not column_name in df.columns:
        raise ValueError(f"No column named {column_name} in dataframe.")
    
    age_list = df[column_name].to_list()
    new_age = []

    for age in age_list:
        if isinstance(age, str):
            n = age.replace(" ", "")
            if 'or' in n:
                new_age.append(int(n[0:2]))
                
            if 'Under' in n:
                new_age.append(int(n[5:7]))
            
            if '-' in n:
                new_age.append((int(n[0:2]) + int(n[3:5]))//2)
                
            if 'Prefer' in n or 'None' in n:
                new_age.append(np.nan)
        
        if isinstance(age, float):
            if np.isnan(age):
                new_age.append(age)
            else:
                new_age.append(round(age))
                
    sum_of_numbers = 0
    length_of_number = 0
    for x in new_age:
        if isinstance(x, int):
            sum_of_numbers += x
            length_of_number += 1 
    mean = round(sum_of_numbers/length_of_number)

    df[column_name].fillna(mean, inplace=True)
  
replace_na_with_mean(df2018, 'age')
replace_na_with_mean(df2019, 'age')
replace_na_with_mean(df2020, 'age')
replace_na_with_mean(df2021, 'age')

df2018['age'] = df2018['age'].apply(clean_age_column)
df2019['age'] = df2019['age'].apply(clean_age_column)
df2020['age'] = df2020['age'].apply(clean_age_column)
df2021['age'] = df2021['age'].apply(clean_age_column)

In [11]:
for df in [df2018, df2019, df2020, df2021]:
    print(df['age'].unique())
# Can not replace None in dataframe with a value

['25 - 30 years old' '37 - 42 years old' '18 - 24 years old'
 '49 - 54 years old' '55 - 60 years old' 'Over 60 years old']
['25 - 30 years old' '18 - 24 years old' '49 - 54 years old'
 '31 - 36 years old' '43 - 48 years old' 'Under 18 years old'
 'Over 60 years old' '55 - 60 years old' '37 - 42 years old']
['31 - 36 years old' '49 - 54 years old' '37 - 42 years old'
 'Under 18 years old' '18 - 24 years old' '25 - 30 years old'
 'Over 60 years old' '43 - 48 years old' '55 - 60 years old']
['37 - 42 years old' '18 - 24 years old' '25 - 30 years old'
 'Over 60 years old' '55 - 60 years old' '49 - 54 years old'
 'Prefer not to say']


In [12]:
for df in [df2018, df2019, df2020, df2021]:
    print(df['gender'].unique())
    break

[None 'Male' 'Female' 'Female;Male' 'Female;Transgender'
 'Non-binary, genderqueer, or gender non-conforming'
 'Male;Non-binary, genderqueer, or gender non-conforming'
 'Female;Male;Transgender;Non-binary, genderqueer, or gender non-conforming'
 'Transgender'
 'Transgender;Non-binary, genderqueer, or gender non-conforming'
 'Female;Non-binary, genderqueer, or gender non-conforming'
 'Female;Transgender;Non-binary, genderqueer, or gender non-conforming'
 'Male;Transgender' 'Female;Male;Transgender'
 'Female;Male;Non-binary, genderqueer, or gender non-conforming'
 'Male;Transgender;Non-binary, genderqueer, or gender non-conforming']


In [45]:

def add_trans_option(df: pd.DataFrame):
    e = []
    for gender, choice in zip(df['gender'].to_list(), df['transgender'].to_list()):
        
        if isinstance(choice, str) and isinstance(gender, str): 
            if 'Yes' in choice:
                e.append(gender +  ';Transgender')
                
            if 'No' in choice:
                e.append(gender)
            
            if 'Prefer not to say' in choice or 'Or, in your own words:' in choice:
                e.append(None)
                
        if isinstance(choice, type(None)) or isinstance(gender, type(None)):
            e.append(gender)
        
    return e
    

In [58]:

df2019['gender'] = add_trans_option(df2019)
df2020['gender'] = add_trans_option(df2020)
df2021['gender'] = add_trans_option(df2021)

# In the 2018 dataset the choices were male and female but in other datasets its man and woman so changing the 2018 dataset index
how = {'Male': 'Man', 'Female': 'Woman'}

df2018['gender'].replace(how, inplace=True)
df2018['gender'].head()

0    None
1    None
2    None
3    None
4    None
Name: gender, dtype: object

In [57]:
df1 = pd.merge(  left_index=True, right_index=True)
df2 = pd.merge(  left_index=True, right_index=True)

dfs_merged = pd.merge(df1, df2, left_index=True, right_index=True)
dfs_merged.columns = ['2018', '2019', '2020', '2021']

dfs_merged

,2018,2019,2020,2021
gender,,,,
Man,59785,78302,46236,74320
None,34386,3477,13904,3503
Woman,4409,6709,4038,4261
"Non-binary, genderqueer, or gender non-conforming",595,1011,624,955
Transgender,423,5970,2700,4120
